# High-dimensional models

This tutorial demonstrates:

* How to use UltraNest in 100 dimensions.
* How to speed up likelihood functions with vectorization
* How to write a program with UltraNest.
* How to execute on multiple cores.


## Model

For simplicity, we integrate a 100-dimensional gaussian:

$$ L = -\frac{1}{2} * \sum^{100}_{i=1} \left(x_i-\frac{1}{2}\right)^2 $$



Solve the problem:

In [ ]:
import ultranest

sampler = ultranest.ReactiveNestedSampler(parameters, vectorized_gauss_likelihood, vectorized=True)

First, we solve the simpler model:

In [ ]:
result = sampler.run(min_num_live_points=400)
sampler.print_results()

Now lets have a go at the harder problem. We limit the number of evaluations:

In [ ]:
result2 = sampler2.run(min_num_live_points=400, max_ncalls=2000000)

The efficiency is very low. This is not just because of the dimensionality of the problem, but also because of the degeneracies. To make progress, lets use a slice sampler:

In [ ]:
import ultranest.stepsampler
# have to choose the number of steps the slice sampler should take
# after first results, this should be increased and checked for consistency.
nsteps = 2 * len(parameters2)
# create step sampler:
sampler2.stepsampler = ultranest.stepsampler.RegionSliceSampler(nsteps=nsteps)
# run again:
result2 = sampler2.run(min_num_live_points=400)
sampler2.print_results()

The efficiency is now constant (at 1/nsteps).

## Plot the parameter posterior probability distribution

A classic corner plot:

In [ ]:
from ultranest.plot import cornerplot
cornerplot(result1)

In [ ]:
cornerplot(result2)

In [ ]:
sampler1.ncall

## Plot the fit

To evaluate whether the results make any sense, we want
to look whether the fitted function goes through the data points.

In [ ]:
plt.figure()
plt.title("1-sine fit")
plt.xlabel('x')
plt.ylabel('y')
plt.errorbar(x=t, y=y, yerr=yerr,
             marker='o', ls=' ', color='orange')


t_grid = np.linspace(0, 5, 400)

from ultranest.plot import PredictionBand
band = PredictionBand(t_grid)

# go through the solutions
for B, A1, P1, t1 in sampler1.results['samples']:
    # compute for each time the y value
    band.add(sine_model1(t_grid, B=B, A1=A1, P1=P1, t1=t1))

band.line(color='k')
# add 1 sigma quantile
band.shade(color='k', alpha=0.3)
# add wider quantile (0.01 .. 0.99)
band.shade(q=0.49, color='gray', alpha=0.2)



In [ ]:
plt.figure()
plt.title("2-sine fit")
plt.xlabel('x')
plt.ylabel('y')
plt.errorbar(x=t, y=y, yerr=yerr,
             marker='o', ls=' ', color='orange')

band = PredictionBand(t_grid)

# go through the solutions
for B, A1, P1, t1, A2, P2, t2 in sampler2.results['samples']:
    # compute for each time the y value
    band.add(sine_model2(t_grid, B=B, A1=A1, P1=P1, t1=t1, A2=A2, P2=P2, t2=t2))

band.line(color='k')
# add 1 sigma quantile
band.shade(color='k', alpha=0.3)
# add wider quantile (0.01 .. 0.99)
band.shade(q=0.49, color='gray', alpha=0.2)





## Model comparison

We now want to know:

**Is the model with 2 components better than the model with one component?**

What do we mean by "better" ("it fits better", "the second component is significant")?

a) Which model is better at predicting data it has not seen yet?

b) Which model is more probably the true one, given this data, and these models (and their parameter spaces)?

c) Which model is simplest, but complex enough to capture the information complexity of the data?


## Bayesian model comparison

Here we will focus on b, and apply Bayesian model comparison. 

For simplicity, we will assume equal a-prior model probabilities.

The Bayes factor is:


In [ ]:
K = np.exp(result2['logz'] - result1['logz'])
K

This tells us, assuming both models are equally probable a-priori, that 
the 2-sine model is 150 times more probable to be the true model than the 1-sine model.

N.B.: Bayes factors are influenced by parameter and model priors. It is a good idea to vary them and see how sensitive the result is. Here, the factor is extremely large, so we can be fairly confident that the 2-sine model is correct.

For making decisions, thresholds are needed. They can be calibrated to desired low false decisions rates with simulations (generate data under the simpler model, look at K distribution).